<a href="https://colab.research.google.com/github/ernanibn/reconhecimento_padroes_23/blob/main/atividade_ensembles.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [102]:
from sklearn.datasets import load_iris

iris = load_iris()

X, y = np.array(iris.data[:, 2:]), np.array(iris.target)

In [103]:
from sklearn.base import BaseEstimator, ClassifierMixin
from collections import Counter
from sklearn.model_selection import cross_validate
import numpy as np
from sklearn.metrics import accuracy_score



def maisFrequente(y):
  return Counter(y.flat).most_common(1)[0][0]

# BaseEstimator, ClassifierMixin classes para executar os scores
class Arvore(BaseEstimator, ClassifierMixin):
  def __init__(self):
    self.caracteristica = 0

  def fit(self, X, y):
    self.valor = int(np.mean(X[:, self.caracteristica]))
    maiores = X[:, self.caracteristica] > self.valor
    if sum(maiores) > 0.0 and sum(~maiores)>0.0:
      self.maiores = Arvore()
      self.maiores.fit(X[maiores, :], y[maiores])
      self.menores = Arvore()
      self.menores.fit(X[~maiores, :], y[~maiores])
    else:
      self.resposta = maisFrequente(y)
      return self

  def predict(self, X):
    y = np.empty((X.shape[0]), dtype=int)
    if hasattr(self, 'resposta'):
      y[:] = self.resposta
    else:
      maiores = X[:, self.caracteristica] > self.valor
      y[maiores] = self.maiores.predict(X[maiores, :])
      y[~maiores] = self.menores.predict(X[~maiores, :])
    return y

modelo = Arvore()
modelo.fit(X, y)
ypred = modelo.predict(X)
accuracy_score(y, ypred), (ypred==y)

from sklearn.ensemble import BaggingClassifier

modelo = BaggingClassifier(Arvore(), n_estimators=100)
results = cross_validate(modelo, X, y)
results['test_score'], np.mean(results['test_score'])

(array([0.96666667, 0.93333333, 0.86666667, 0.93333333, 0.93333333]),
 0.9266666666666667)